<a href="https://colab.research.google.com/github/AndoniFikri/Credit-Risk-Prediction-with-Deployment/blob/main/Model_Training_and_Evaluation_(DTS_Final_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns=None
pd.options.display.max_rows=None

#Import Data Set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Set/credit risk cleaned.csv')

In [ ]:
df.info()

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.info()

#Data Split Feature/target and Scalling

In [ ]:
X = df.drop('loan_status', axis=1)
Y = df['loan_status']

In [ ]:
#Check for Data Balanced/Imbalanced
Y.value_counts()/len(Y)*100

Quite Imbalanced, perform SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_res, y_res = smote.fit_resample(X, Y)

In [ ]:
#Train Test Split
#Split data 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

#Model Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score

model_alg = [RandomForestClassifier(), 
             DecisionTreeClassifier(),
             KNeighborsClassifier(), 
             LogisticRegression() 
             ]


In [ ]:
df_mod_eval = pd.DataFrame(columns = ['Algorithm','AUC Score'])
for model in model_alg : 
  model.fit(X_train, y_train)
  algorithm = str(type(model)).split('.')[-1][:-2]
  y_pred_proba = model.predict_proba(X_test)[::,1]
  auc = roc_auc_score(y_test, y_pred_proba)
  df_mod_eval = df_mod_eval.append({'Algorithm' : algorithm, 
                                    'AUC Score' : auc},ignore_index=True)
df_mod_eval

#Model Evaluation

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train) 

In [ ]:
from sklearn.metrics import classification_report
y_pred = rfc.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
#Confusion Matrix 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(10,7))
target_pred = rfc.predict(X_test)

cm = confusion_matrix(y_test, target_pred, labels=rfc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=rfc.classes_)
disp.plot(ax=ax)
plt.show()

In [ ]:
X_train.shape